# NOTE: OLD NOTEBOOK - KEPT FOR REFERENCE - SEE NEWER NOTEBOOK

In [1]:
import pandas as pd

# Feature Vector Processing
This notebook is dedicatd to processing the feature vector outputs from FEATURE.

FEATURE outputs are generally a a count of nearby items within a specified shell. Therefore, you can sum each atom together and get the total microenvironment around a residue.
- Don't want to just use the C-alpha because that would loose important microenvironment information for large, bulky residues.

### Things that need to be done:

- [x] Create feature vector header parser (Need for 1 shell and extension to n shells)
- [x] Collapse multiple atoms into a single residue value

## Processing Functions and Storage

In [45]:
### Basic Column Headers ###
column_headers = []
fp = open('feature-vector-comments.md')
for line in fp:
    header_item = [item.replace(',','').lower() for item in line.split()]
    header_item.remove('properties')
    column_headers.extend(header_item)
fp.close()

### Valid Residues ###
valid_resid = {'ala', 'arg', 'asn', 'asp',
               'cys', 'gln', 'glu', 'gly',
               'his', 'ile', 'leu', 'lys',
               'met', 'phe', 'pro', 'ser',
               'thr', 'trp', 'tyr', 'val'
               }

### Mapping from Protein Name to PDB Structure ###
prot_pdb_dict = {'TEM-1': '1btl',
                 'Kka2': '1nd4',
                 'Uba1': '6dc6',
                 'PSD95pdz3': '1TP3',
                 'Pab1': '1IFW',
                 'Yap65': '1jmq',
                 'hsp90': '2xjx',
                 'gb1': '2qmt'
                }

def extend_columns(n):
    """ Duplicates basic columns and adds shell number indicator
    """

    final_headers = []
    for i in range(n):
        temp_headers = [header + '_' + str(i + 1) for header in column_headers]
        final_headers.extend(temp_headers)
    return(final_headers)

def gen_headers(n):
    """ Generates list of pandas headers for FEATURE vector data
    """

    new_headers = ['env_name']
    extend_headers = extend_columns(n)
    new_headers.extend(extend_headers)
    new_headers.append('residue')
    return(new_headers)

def drop_coordinates(df):
    """ Drops coordinate/position columns from FEATURE vector data
    """

    num_cols = len(df.columns)
    start = num_cols - 6
    stop = num_cols - 1
    cols_to_delete = list(range(start,stop))
    df.drop(cols_to_delete, axis = 1, inplace = True)

def get_pdb(row):
    """ Extracts PDB from environment description column
    """

    env_name = row['env_name']
    if '1btl' in env_name:
        return('1btl')
    if '1nd4' in env_name:
        return('1nd4')
    if '6dc6' in env_name:
        return('6dc6')
    if '1tp3' in env_name:
        return('1tp3')
    if '1ifw' in env_name:
        return('1if2')
    if '1jmq' in env_name:
        return('1jmq')
    if '2xjx' in env_name:
        return('2xjx')
    if '2qmt' in env_name:
        return('2qmt')

def prot_to_pdb(row):
    """ Maps protein identifier to PDB accession number to create new column
    """

    return(prot_pdb_dict[row['protein']])

def get_resnum(row):
    """ Extracts residue number from the res column in FEATURE vector data
    """

    value = row['res']
    return(int(value[3:]))

def get_resid(row):
    """ Extracts residue ID from res column in FEATURE vector data
    """

    value = row['res']
    return(value[:3].lower())

## Clean atom string
def clean_atom(row):
    """ Removes extraneous characters from atom description
    """
    
    return(row['atom'][2:])

## Single Shell (pdb-shell1.ff)

In [46]:
## Basic Processing
shell1_path = 'data/feature-files/pdb-shell1.ff'
shell1 = pd.read_csv(shell1_path, sep='\t', header=None)
drop_coordinates(shell1)
shell1.columns = gen_headers(1)

# Generate better descriptive columns
shell1['pdb'] = shell1.apply(lambda row: get_pdb(row), axis=1)
shell1[['res', 'atom']] = shell1.residue.str.split(":",expand=True)
shell1['resnum'] = shell1.apply(lambda row: get_resnum(row), axis=1)
shell1['resid'] = shell1.apply(lambda row: get_resid(row), axis=1)
shell1['atom'] = shell1.apply(lambda row: clean_atom(row), axis=1)

In [47]:
# Filtering and Aggregating
shell1_agg = shell1.groupby(['pdb', 'resnum', 'resid']).sum().reset_index()
shell1_filt = shell1_agg[shell1_agg.resid.isin(valid_resid)]

In [48]:
shell1_filt.resid.unique()

array(['his', 'pro', 'glu', 'thr', 'leu', 'val', 'lys', 'asp', 'ala',
       'gln', 'gly', 'arg', 'tyr', 'ile', 'asn', 'ser', 'phe', 'met',
       'cys', 'trp'], dtype=object)

In [97]:
## Double checking Residue Alignment

## TEM-1
proteins = ['1btl', '1nd4', '6dc6', '1tp3', '1ifw', '1jmq', '2xjx', '2qmt']

shell1_filt.pdb == '2qmt'

# for protein in proteins:
#     position = sorted(shell1_filt[shell1_filt.pdb == protein].resnum.unique())
#     print(protein, min(position), max(position))

0       False
1       False
2       False
3       False
4       False
        ...  
3109    False
3110    False
3111    False
3112    False
3113    False
Name: pdb, Length: 2136, dtype: bool

## 6 Shell (Default) (pdb-shell4.ff)

In [49]:
### Basic Data Processing ###

shell6_path = 'data/feature-files/pdb-shell6.ff'
shell6= pd.read_csv(shell6_path, sep='\t', header=None)
drop_coordinates(shell6)
shell6.columns = gen_headers(6)

shell6['pdb'] = shell6.apply(lambda row: get_pdb(row), axis=1)
shell6[['res', 'atom']] = shell6.residue.str.split(":",expand=True)
shell6['resnum'] = shell6.apply(lambda row: get_resnum(row), axis=1)
shell6['resid'] = shell6.apply(lambda row: get_resid(row), axis=1)
shell6['atom'] = shell6.apply(lambda row: clean_atom(row), axis=1)

In [50]:
### Filtering and Aggregation ###
shell6_agg = shell6.groupby(['pdb', 'resnum', 'resid']).sum().reset_index()
shell6_filt = shell6_agg[shell6_agg.resid.isin(valid_resid)]

# Merging Features

In this feature merge, I assume that the calculation of the local microenvironments calculated by FEATURE will be identical for each mutation at a specific residue. While not strictly true due to mutations introducing *some* changes in the environment, this is the best model without relying on molecular simulation to relax mutated structures (see Rosetta).

In [55]:
# Load Dataset 
main_data_path = 'data/mmc2.csv'
main_data = pd.read_csv(main_data_path)
main_data['pdb'] = main_data.apply(lambda row: prot_to_pdb(row), axis=1)
main_data.rename(columns = {'position':'resnum'}, inplace=True)

In [56]:
## Removing Redundant or Bad Columns
main_data.drop(['local_density', 'accessibility','local_biochem', 'mut_mut_msa_congruency'],axis=1, inplace=True)
main_data.drop(['variant_id', 'position_id', 'dms_id', 'uniprot', 'predicted?'], axis=1, inplace=True)
main_data.dropna(subset=['scaled_effect'], inplace=True)
main_data.dropna(subset=['aa2_psic'], inplace=True)
main_data['dssp_sec_str'].replace('.', 'O', inplace=True)
main_data['dssp_sec_str'].fillna('O', inplace=True)
main_data['phi_psi_reg'].fillna('O', inplace=True)
main_data['wt_mut'].fillna('NA', inplace=True)
main_data.fillna(0, inplace=True)

In [57]:
main_data.protein.unique()

array(['TEM-1', 'Kka2', 'Uba1', 'PSD95pdz3', 'Pab1', 'Yap65', 'hsp90',
       'gb1'], dtype=object)

In [58]:
main_update = pd.get_dummies(main_data,columns=['aa1', 'aa2', 'wt_mut', 'aa1_polarity', 'aa2_polarity', 'dssp_sec_str', 'phi_psi_reg'])

In [73]:
# main_update.info(verbose=True)
main_update.protein.unique()
main_update.shape
main_update.pdb.unique()

array(['1btl', '1nd4', '6dc6', '1TP3', '1IFW', '1jmq', '2xjx', '2qmt'],
      dtype=object)

In [69]:
shell1_filt.pdb.unique()

array(['1btl', '1if2', '1jmq', '1nd4', '1tp3', '2qmt', '2xjx', '6dc6'],
      dtype=object)

In [84]:
# Merge FEATURE Vectors into main dataset
merged_shell1 = pd.merge(shell1_filt, main_update, on=['pdb', 'resnum'], how='right')
# merged_shell6 = pd.merge(main_update,shell6_filt, on=['pdb', 'resnum'])

In [79]:
print(merged_shell1.protein.unique())
print(merged_shell6.protein.unique())
merged_shell1.shape
merged_shell1.pdb.unique()

['TEM-1' 'Kka2' 'Uba1' 'PSD95pdz3' 'Pab1' 'Yap65' 'hsp90' 'gb1']
['TEM-1' 'Kka2' 'Uba1']


array(['1btl', '1nd4', '6dc6', '1TP3', '1IFW', '1jmq', '2xjx', '2qmt'],
      dtype=object)

In [85]:
merged_shell1.to_csv('data/merged_shell1.csv')
merged_shell6.to_csv('data/merged_shell6.csv')